In [1]:
from sublimpy import utils, variables, tidy
import pandas as pd
import altair as alt
import numpy as np

In [29]:
stossel_df = pd.read_csv("Stossel_box_weights - Sheet1.csv")[['date_time', 'delta_box_1', 'delta_box_2']]
stossel_df['time'] = pd.to_datetime(stossel_df['date_time'])
stossel_df['time_previous'] = stossel_df.time.shift(1)
stossel_df

,date_time,delta_box_1,delta_box_2,time,time_previous
0,20230131 18:45:00,NaN,NaN,2023-01-31 18:45:00,NaT
1,20230201 07:55:00,-11.0,285.0,2023-02-01 07:55:00,2023-01-31 18:45:00
2,20230201 16:30:00,-1.0,-19.0,2023-02-01 16:30:00,2023-02-01 07:55:00
3,20230202 07:57:00,34.0,NaN,2023-02-02 07:57:00,2023-02-01 16:30:00
4,20230202 17:33:00,-21.0,NaN,2023-02-02 17:33:00,2023-02-02 07:57:00
5,20230202 17:40:00,NaN,NaN,2023-02-02 17:40:00,2023-02-02 17:33:00
6,20230203 07:58:00,-20.0,20.0,2023-02-03 07:58:00,2023-02-02 17:40:00
7,20230203 16:26:00,0.0,-23.0,2023-02-03 16:26:00,2023-02-03 07:58:00
8,20230204 07:55:00,21.0,21.0,2023-02-04 07:55:00,2023-02-03 16:26:00
9,20230204 16:30:00,-48.0,-49.0,2023-02-04 16:30:00,2023-02-04 07:55:00


In [30]:
line_base = alt.Chart().transform_calculate(
    a = '0'
).mark_rule()

alt.Chart(stossel_df).mark_circle().encode(
    alt.X("delta_box_1:Q").scale(domain=[-100, 100]),
    alt.Y("delta_box_2:Q").scale(domain=[-100, 100])
) + line_base.encode(x='a:Q') + line_base.encode(y='a:Q')

alt.LayerChart(...)

In [31]:
# filter outliers
stossel_df = stossel_df[np.abs(stossel_df.delta_box_1) < 70]
stossel_df = stossel_df[np.abs(stossel_df.delta_box_2) < 70]
stossel_df = stossel_df[np.abs(stossel_df.delta_box_1) > 5]
stossel_df = stossel_df[np.abs(stossel_df.delta_box_2) > 5]
# remove points that disagree in terms of sign
stossel_df = stossel_df[
    ((stossel_df.delta_box_1 > 0) & (stossel_df.delta_box_2 > 0))
    |
    ((stossel_df.delta_box_1 < 0) & (stossel_df.delta_box_2 < 0))
]

In [32]:
alt.Chart(stossel_df).mark_circle().encode(
    alt.X("delta_box_1:Q").scale(domain=[-100, 100]),
    alt.Y("delta_box_2:Q").scale(domain=[-100, 100])
) + line_base.encode(x='a:Q') + line_base.encode(y='a:Q')

alt.LayerChart(...)

In [33]:
stossel_df['delta_box_mean'] = np.mean([stossel_df['delta_box_1'], stossel_df['delta_box_2']], axis=0)
stossel_df['delta_box_mean_mm'] = 1000 * stossel_df['delta_box_mean'] / 997000  # 1000(mm/m)*(∆mass in g/m^2) / (h2o density in g/m^3) ==> 

In [34]:
line_base = alt.Chart().transform_calculate(
    a = '0'
).mark_rule()
alt.Chart(stossel_df).mark_circle().encode(
    alt.X("delta_box_1:Q").scale(domain=[-100, 100]),
    alt.Y("delta_box_2:Q").scale(domain=[-100, 100])
) + line_base.encode(x='a:Q') + line_base.encode(y='a:Q')

alt.LayerChart(...)


50 grams of water sublimated

there are 997000 g/m^3 of water.

That means 50 / 997000 is the volume of water that sublimated.

In [35]:
stossel_df['daytime'] = stossel_df['time'].dt.hour > 12

In [36]:
line_base = alt.Chart().transform_calculate(
    a = '0'
).mark_rule()
alt.Chart(stossel_df).mark_circle().encode(
    alt.X("delta_box_1:Q").scale(domain=[-100, 100]),
    alt.Y("delta_box_2:Q").scale(domain=[-100, 100]),
    alt.Color("daytime:N")
) + line_base.encode(x='a:Q') + line_base.encode(y='a:Q')


alt.LayerChart(...)

In [39]:
to_output = stossel_df[['time_previous', 'time', 'delta_box_mean_mm', 'daytime']]
to_output

,time_previous,time,delta_box_mean_mm,daytime
8,2023-02-03 16:26:00,2023-02-04 07:55:00,0.021063,False
9,2023-02-04 07:55:00,2023-02-04 16:30:00,-0.048646,True
10,2023-02-04 16:30:00,2023-02-05 08:02:00,0.013541,False
13,2023-02-07 12:28:00,2023-02-07 17:28:00,-0.044634,True
15,2023-02-09 17:00:00,2023-02-10 08:00:00,0.015547,False
16,2023-02-10 08:00:00,2023-02-10 17:30:00,-0.019057,True
17,2023-02-10 17:30:00,2023-02-11 08:00:00,0.028586,False
18,2023-02-11 08:00:00,2023-02-11 17:30:00,-0.037111,True
20,2023-02-12 07:15:00,2023-02-12 17:10:00,-0.039117,True
21,2023-02-12 17:10:00,2023-02-13 07:50:00,0.008526,False


In [ ]:
to_output.to_csv("processed_lysimeter_data.csv")

# Get sos data

In [ ]:
sos_ds = utils.download_sos_data(
    stossel_df['time'].min().strftime('%Y%m%d'),
    stossel_df['time'].max().strftime('%Y%m%d'),
    variable_names=variables.DEFAULT_VARIABLES,
    local_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc/',
    cache=True
)

In [ ]:
tidy_df = tidy.get_tidy_dataset(sos_ds, list(sos_ds.data_vars))

## Calculate daily sublimation with the sos datasets, exclude measurements from hours 18-7

In [ ]:
daily_sublimation_df = tidy_df[tidy_df['variable'] == 'w_h2o__3m_c'][(tidy_df.time.dt.hour > 7) & (tidy_df.time.dt.hour < 18)].set_index('time').groupby([pd.Grouper(freq='24H'), 'variable', 'height', 'tower', 'measurement']).mean().reset_index()
daily_sublimation_df['value'] = daily_sublimation_df['value']*11*60*60
daily_sublimation_df['value'] = 1000 * daily_sublimation_df['value'] / 997000  # 1000(mm/m)*(∆mass in g/m^2) / (h2o density in g/m^3) ==> 

# Clean stossel data - remove nighttime measurements

In [ ]:
stossel_df = stossel_df[pd.to_datetime(stossel_df['date_time']).dt.hour > 12]
stossel_df['time'] = stossel_df['time'].apply(lambda dt: dt.replace(hour=0, minute=0))
stossel_df = stossel_df.assign(
        measurement = 'manual lysimeter',
        delta_box_mean_mm = - stossel_df['delta_box_mean_mm']
).rename(columns={'delta_box_mean_mm':'value'})

In [ ]:
src = pd.concat([
    stossel_df[['time', 'measurement', 'value']],
    daily_sublimation_df[['time', 'measurement', 'value']]
])

time_series_bar_chart = alt.Chart(src).mark_bar().encode(
        alt.X("date(time):O").title("Day of February"),
        alt.Y("value:Q").title("Sublimation (SWE, mm)"),
        alt.XOffset("measurement:N"),
        alt.Color("measurement:N")
    ).properties(width=600)

scatter_chart = alt.Chart(stossel_df.assign(day = stossel_df.time.dt.day)).mark_point(size=50).encode(
    alt.X("delta_box_1:Q").scale(domain=[-60, 5]).title("Box 1 (g/m^2)"),
    alt.Y("delta_box_2:Q").scale(domain=[-60, 5]).title("Box 2 (g/m^2)"),
    alt.Shape("day:N").title("Day of February")
)

onetoone_line_chart =  alt.Chart(pd.DataFrame({
    'x':np.linspace(-60,5, 10), 
    'y':np.linspace(-60,5, 10)
})).mark_line(color='grey').encode(
    x = 'x',
    y = 'y'
)

(
    time_series_bar_chart.properties(height = 150,width=300) | (onetoone_line_chart + scatter_chart).properties(height = 150, width=150)
).resolve_scale(color='independent', shape='independent').configure_legend(orient='top', columns=4)

In [ ]:
src.head()

In [ ]:
alt.Chart(src).mark_bar().encode(
        alt.X("date(time):O").title("Day of February"),
        alt.Y("value:Q").title("Sublimation (SWE, mm)"),
        alt.XOffset("measurement:N"),
        alt.Color("measurement:N")
    ).properties(width=600)